In [ ]:
import pandas as pd
import numpy as np

from utils.dataset import capture_screenshots_async

In [ ]:
# create a dataset
# get screenshot of webpage of provided URL and save screenshot
# add path in 'img_path' column

data = pd.read_excel("./data/Price_Normalization_Dataset.xlsx")

dataset = await capture_screenshots_async(data)

cols_to_fix = ["Explictness","Step1_desired_unit", "Step1_desired_value"]

dataset[cols_to_fix] = dataset[cols_to_fix].where(
    pd.notna(dataset[cols_to_fix]),
    None
)
dataset.to_excel("./dataset/dataset.xlsx")

### STEP1
#### Infer quantity and unit from webpage, with evidence

In [ ]:
# pydantic models
# step1output model

from pydantic import BaseModel
from typing import Optional, Literal, List

class Evidence(BaseModel):
    source: Literal["price_label", "specification_table", "more_information","calculator box"]
    text: str

class Quantity(BaseModel):
    value: float
    unit: str


class Step1Output(BaseModel):
    priced_quantity: Optional[Quantity]
    explicitness: Literal["direct", "indirect", "none"]
    confidence: float
    evidence: List[Evidence]
    notes: str


In [ ]:
SYSTEM_PROMPT = """You are a pricing quantity inference engine.

Your task is to identify the quantity that the listed price applies to,
using ONLY explicit seller-visible information from the webpage.

Allowed evidence:
- Price labels or text directly adjacent to the price
- Specification tables
- Explicit product labels stating quantity or unit

Do NOT:
- infer from typical product sizes
- assume standard lengths
- perform unit conversion
- perform calculations
- if unit is not visible do not infer or assume any unit
- treat cart quantity selectors, add-to-cart counters, or default quantity values (e.g. "Quantity: 1") as a pricing unit.

Do:
- Unit must be in full from
- Convert millimeter to meters
- If unit is not visible directlty adjacent to main price label set explicitness to "indirect"
- If no standard unit (length, weight, area, volume, pack size) is explicitly stated,
set explicitness to "none".

Return ONLY a JSON object that matches the provided schema.
Billing accuracy is required."""


USER_PROMPT = 'screenshot : '

In [ ]:
# load model
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv()

model = ChatOpenAI(model="gpt-5.2")

In [ ]:
import base64
from langchain_core.messages import HumanMessage, SystemMessage
from openai import OpenAIError


model_with_structured_output = model.with_structured_output(Step1Output)


# llm function
async def infer_from_webpage(image_path : str) -> Step1Output:

    '''input : an image'''

    try:
        with open(image_path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

            human_msg = HumanMessage([
                    {"type": "text", "text": USER_PROMPT},
                    {
                        "type": "image",
                        "base64": encoded_image,
                        "mime_type": "image/png",
                    },
                ])
            
            system_msg = SystemMessage(SYSTEM_PROMPT)

            response = await model_with_structured_output.ainvoke([system_msg,human_msg])

            return response
        
    except OpenAIError as e:
        raise RuntimeError(f"OpenAI API error: {e}") from e

    except Exception as e:
        raise RuntimeError(f"Inference failed: {e}") from e


In [ ]:
# sample record
import pandas as pd
dataset = pd.read_excel('./dataset/dataset.xlsx')
dataset = dataset.rename(columns={'Unit\n':'TargetUnit','Rate £':'ExtractedPrice'})

sample = dataset.loc[3]

In [ ]:
# unit test infer_from_webpage
image_path = sample['img_path']

response = await infer_from_webpage(image_path)
response

In [ ]:
# function to run on multiple records
# input a dataset dataframe
# iterate and perform infer from webpage

import asyncio
import pandas as pd

async def run_step1_on_dataset(
    df: pd.DataFrame,
    semaphore_limit: int = 5,
) -> pd.DataFrame:
    

    df = df.copy()
    df["Output"] = None

    semaphore = asyncio.Semaphore(semaphore_limit)


    async def run_one(index, row):
        async with semaphore:
            try:
                output: Step1Output = await infer_from_webpage(
                    image_path=row["img_path"],
                )
                return index, output.model_dump_json()

            except Exception as e:
                # Store error as string (important for dataset audit)
                return index, f"ERROR: {str(e)}"

    tasks = [
        run_one(idx, row)
        for idx, row in df.iterrows()
    ]

    results = await asyncio.gather(*tasks)

    for idx, result in results:
        df.at[idx, "Output"] = result

    return df

In [ ]:
# test on multiple records

dataset = pd.read_excel("./dataset/dataset.xlsx")
result_df = await run_step1_on_dataset(dataset[-2:])
result_df.to_excel('./result/result5.xlsx')

## Evaluation

### STEP2

In [ ]:
# initialize model

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
load_dotenv()

model = ChatOpenAI(model="gpt-5.2")

In [ ]:
# define pydantic model for output

from pydantic import BaseModel
from typing import Optional, Literal, List


class Step2Output(BaseModel):
    status: Literal["ok", "fail"]

    expression: Optional[str]
    """
    Mathematical expression using variable `price`,
    e.g. '{price} * (1000 / 3.85)'
    """

    explanation: Optional[str]
    """
    Short factual justification describing which explicit evidence
    was used to form the expression.
    Must NOT include calculations or algebraic transformations.
    """

    evidence_used: List[str]
    """
    Exact seller-visible texts or table rows used
    (e.g. 'Weight/Metre (Kg): 3.85')
    """


In [ ]:
# create structured ouptut model

structured_model = model.with_structured_output(Step2Output)

In [ ]:
import ast
import operator as op


# Allowed operators
_ALLOWED_OPERATORS = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
}


class MathExpressionError(Exception):
    pass


def evaluate_math_expression(expression: str) -> float:
    """
    Safely evaluate a mathematical expression consisting of
    numbers, + - * / and parentheses.

    Args:
        expression (str): math expression, e.g. "(1000 / 3.85)"

    Returns:
        float: evaluated result

    Raises:
        MathExpressionError: if expression is invalid or unsafe
    """

    try:
        parsed = ast.parse(expression, mode="eval")
        return _eval_node(parsed.body)

    except Exception as e:
        raise MathExpressionError(f"Invalid math expression: {expression}") from e


def _eval_node(node):
    if isinstance(node, ast.Num):  # Python <3.8
        return node.n

    if isinstance(node, ast.Constant):  # Python 3.8+
        if isinstance(node.value, (int, float)):
            return node.value
        raise MathExpressionError("Only numeric constants allowed")

    if isinstance(node, ast.BinOp):
        if type(node.op) not in _ALLOWED_OPERATORS:
            raise MathExpressionError(f"Operator {type(node.op)} not allowed")

        left = _eval_node(node.left)
        right = _eval_node(node.right)

        return _ALLOWED_OPERATORS[type(node.op)](left, right)

    if isinstance(node, ast.UnaryOp):
        if isinstance(node.op, ast.USub):
            return -_eval_node(node.operand)
        raise MathExpressionError("Unary operator not allowed")

    raise MathExpressionError(f"Unsupported expression element: {type(node)}")


In [ ]:
import re
from typing import Dict


def execute_llm_expression(
    raw_expression: str,
    variables: Dict[str, float],
) -> float:
    """
    Execute a math expression produced by the LLM.

    Steps:
    1. Template variables (price -> {price})
    2. Substitute runtime values
    3. Validate expression safety
    4. Evaluate using safe math executor

    Args:
        raw_expression: e.g. "price * (1000 / 0.89)"
        variables: e.g. {"price": 10}

    Returns:
        float result

    Raises:
        ValueError if expression is unsafe or invalid
    """

    # --- Step 1: template variables ---
    for var in variables.keys():
        raw_expression = re.sub(
            rf"\b{var}\b",
            f"{{{var}}}",
            raw_expression
        )

    # --- Step 2: substitute variables ---
    expr = raw_expression
    for key, value in variables.items():
        if not isinstance(value, (int, float)):
            raise ValueError(f"Invalid value for {key}: {value}")
        expr = expr.replace(f"{{{key}}}", str(value))

    # --- Step 3: ensure no unresolved placeholders remain ---
    if re.search(r"\{[a-zA-Z_]+\}", expr):
        raise ValueError(f"Incorrect Math Exp: {expr}")

    # --- Step 4: validate allowed characters only ---
    if not re.fullmatch(r"[0-9\.\+\-\*\/\(\)\s]+", expr):
        raise ValueError(f"Incorrect Math Exp: {expr}")

    # --- Step 5: evaluate using Step-3 calculator ---
    return evaluate_math_expression(expr)


### STEP3

PROMPTS

In [ ]:
# Version1 :  single system prompt 
  
SYSTEM_PROMPT = """You are a pricing conversion engine.

Your task is to generate a MATHEMATICAL EXPRESSION to convert given unit price to target unit price
from the priced quantity unit to the target unit.

You are provided with:
- a screenshot of webpage of a product
- "target_unit"
- a user "comment" might be helpful to derive a MATHEMATICAL EXPRESSION

<INSTRUCTIONS>
- Infer the quanity and given unit of the product from the webpage
- If the given unit and target unit is not same, identify the conversion factor from webpage. For example the qiven quantity is in liter and target quantity in Kg, check for denisty is the specs or more info table
- If the given unit and target unit is same, mathematical experssion is straight forward
- use "comment" when no unit or quanity or conversion factor cannot to be determined from webpage


STRICT RULES:
- Do NOT assume material properties or typical values.
- Do NOT restate steps.
- Do NOT calculate numeric results.
- Do NOT algebraically rewrite or invert expressions.
- Use ONLY numeric values explicitly visible in the screenshot or evidence.
- Do NOT assume material properties.
- Do NOT invent constants.
- If the required conversion factor is not explicitly visible, return FAIL.

VARIABLE RULES:
- Use the variable name `price` for the price that will be substituted
- All other values must be numeric literals taken directly from the screenshot.

ALLOWED OPERATIONS:
- +  -  *  /
- parentheses ( )


- return a valid math expression with variable "price" (e.g. price * (1000 / 3.85)`), OR
- the single word: FAIL

"""

In [ ]:
# Version2 :  two systems prompts;switch to prompt that emphasize "Comments" if information cannot be inferred from webpage
  
SYSTEM_PROMPT_SELLER = """You are a pricing conversion engine.

Your task is to generate a MATHEMATICAL EXPRESSION to convert given unit price to target unit price
from the priced quantity unit to the target unit.

You are provided with:
- a screenshot of webpage of a product
- "target_unit"
- a user "comment" might be helpful to derive a MATHEMATICAL EXPRESSION

<INSTRUCTIONS>
- Infer the quanity and given unit of the product from the webpage
- If the given unit and target unit is not same, identify the conversion factor from webpage. For example the qiven quantity is in liter and target quantity in Kg, check for denisty is the specs or more info table
- If the given unit and target unit is same, mathematical experssion is straight forward
- use "comment" when no unit or quanity or conversion factor cannot to be determined from webpage


STRICT RULES:
- Do NOT assume material properties or typical values.
- Do NOT restate steps.
- Do NOT calculate numeric results.
- Do NOT algebraically rewrite or invert expressions.
- Use ONLY numeric values explicitly visible in the screenshot or evidence.
- Do NOT assume material properties.
- Do NOT invent constants.
- If the required conversion factor is not explicitly visible, return FAIL.

VARIABLE RULES:
- Use the variable name `price` for the price that will be substituted
- All other values must be numeric literals taken directly from the screenshot.

ALLOWED OPERATIONS:
- +  -  *  /
- parentheses ( )


- return a valid math expression with variable "price" can be subtituted in python math evaulator (e.g. `{/{price}/} * (1000 / 3.85)`), OR
- the single word: FAIL

"""


SYSTEM_PROMPT_COMMENT = """You are a pricing conversion engine operating in COMMENT-PRIMARY mode.

The webpage does NOT provide an explicit conversion between the priced unit
and the target unit.

Your task is to generate a MATHEMATICAL EXPRESSION using the user-provided
comment as the PRIMARY source of quantity or conversion information.

You are provided with:
- target_unit
- a user comment

RULES:
- Treat the comment as a claim about how the product is sold.
- Validate that the comment does NOT contradict visible product details
  (dimensions, material type, product category).
- Do NOT assume typical sizes or industry standards.
- If the comment is ambiguous or contradicts product details, return FAIL.
- You may use simple arithmetic implied directly by the comment
  (e.g. "price is for 5m" → divide by 5).

STRICT CONSTRAINTS:
- Do NOT invent numeric values not present in the comment.
- Do NOT calculate final numeric results.
- Do NOT rewrite or invert expressions.
- Use variable `price` only.

OUTPUT:
Return a JSON object matching the schema.
If conversion cannot be safely derived, return FAIL.

Billing accuracy is required."""




In [ ]:
# set logger
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
)

logger = logging.getLogger("__info__")


In [ ]:
# Version2 : 

from langchain_core.messages import HumanMessage, SystemMessage
import asyncio
import base64
import pandas as pd


async def run_step2_with_retry(
    *,
    structured_model,
    encoded_image: str,
    row,
    max_retries: int = 1,
):
    """
    Try seller-evidence prompt first.
    Retry once with comment-primary prompt if needed.
    """

    attempts = [
        ("SCREENSHOT", SYSTEM_PROMPT_SELLER),
        ("COMMENT", SYSTEM_PROMPT_COMMENT),
    ]

    last_error = None

    for attempt_name, system_prompt in attempts[: max_retries + 1]:
        try:
            user_prompt = (
                f"Comment: {row.get('Comments')}\n"
                f"Target Unit: {row.get('TargetUnit')}\n"
                f"Screenshot:"
            )

            human_msg = HumanMessage(
                [
                    {"type": "text", "text": user_prompt},
                    {
                        "type": "image",
                        "base64": encoded_image,
                        "mime_type": "image/png",
                        "detail": "low",
                    },
                ]
            )

            system_msg = SystemMessage(system_prompt)

            response: Step2Output = await structured_model.ainvoke(
                [system_msg, human_msg]
            )

            if response.status == "ok" and response.expression:
                logger.info(
                    "STEP2_SUCCESS | mode=%s | expr=%s",
                    attempt_name,
                    response.expression,
                )
                return response, attempt_name, None

            last_error = f"{attempt_name}_INVALID_OUTPUT"
            

        except Exception as e:
            last_error = f"{attempt_name}_ERROR: {e}"
            logger.error(last_error)

    return response, attempt_name, last_error


async def run_step2_and_3_on_dataset_v2(
    dataset: pd.DataFrame,
    structured_model,
    semaphore_limit: int = 5,
) -> pd.DataFrame:

    semaphore = asyncio.Semaphore(semaphore_limit)

    dataset = dataset.copy()
    dataset["final_unit_price"] = None
    dataset["final_expression"] = None
    dataset["explanation"] = None
    dataset["error"] = None
    dataset["conversion_mode"] = None

    async def process_row(idx: int, row: pd.Series):
        async with semaphore:
            try:
                extracted_price = float(row["ExtractedPrice"])
                image_path = row["img_path"]

                # ---- encode image ----
                with open(image_path, "rb") as f:
                    encoded_image = base64.b64encode(f.read()).decode("utf-8")

                # ---- Step 2 (with retry) ----
                response, mode, error = await run_step2_with_retry(
                    structured_model=structured_model,
                    encoded_image=encoded_image,
                    row=row,
                )

                

                if error:
                    dataset.at[idx, "error"] = error
        

                expr = response.expression
                explanation = response.explanation

                response_json = response.model_dump()

                logger.info(
                            "LLM_RESPONSE | row=%s | response=%s | comment=%s",
                            idx,
                            response_json,
                            row["Comments"]
                            )

                # ---- Step 3 (execution) ----
                try:
                    result = execute_llm_expression(
                        raw_expression=expr,
                        variables={"price": extracted_price},
                    )
                except Exception as exec_error:
                    dataset.at[idx, "error"] = f"EXECUTION_ERROR: {exec_error}"
                    dataset.at[idx, "final_expression"] = expr
                    dataset.at[idx, "explanation"] = explanation
                    return

    
                dataset.at[idx, "final_unit_price"] = result
                dataset.at[idx, "final_expression"] = expr
                dataset.at[idx, "explanation"] = explanation
                dataset.at[idx, "conversion_mode"] = mode

                logger.info(
                    "ROW_SUCCESS | row=%s | mode=%s | result=%s",
                    idx,
                    mode,
                    result,
                )

            except Exception as fatal_error:
                logger.exception("FATAL_ERROR | row=%s", idx)
                dataset.at[idx, "error"] = f"FATAL_ERROR: {fatal_error}"

    tasks = [
        process_row(idx, row)
        for idx, row in dataset.iterrows()
    ]

    await asyncio.gather(*tasks)

    return dataset




In [ ]:
# Version 1 :

import asyncio
import base64
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage


async def run_step2_and_3_on_dataset(
    dataset: pd.DataFrame,
    structured_model,
    system_prompt: str,
    semaphore_limit: int = 5,
) -> pd.DataFrame:
    """
    Runs Step-2 (LLM expression generation) + Step-3 (execution)
    on each row of the dataset asynchronously.

    Required columns in dataset:
    - img_path
    - ExtractedPrice
    """

    semaphore = asyncio.Semaphore(semaphore_limit)

    # ensure output columns exist
    dataset = dataset.copy()
    dataset["final_unit_price"] = None
    dataset["final_expression"] = None
    dataset["explanation"] = None
    dataset["error"] = None

    async def process_row(idx: int, row: pd.Series):
        async with semaphore:
            try:
                image_path = row["img_path"]
                extracted_price = float(row["ExtractedPrice"])


                USER_PROMPT = """Comment : {Comment} \nTarget Unit : {TargetUnit}\n
                                Screenshot : """
                
                llm_input = {
                            "Comment": row.get("Comments"),
                            "TargetUnit": row.get("TargetUnit"),
                            }

                prompt = USER_PROMPT.format(**llm_input)

                # --- encode image ---
                with open(image_path, "rb") as f:
                    encoded_image = base64.b64encode(f.read()).decode("utf-8")

                human_msg = HumanMessage(
                    [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image",
                            "base64": encoded_image,
                            "mime_type": "image/png",
                            "detail": "low",
                        },
                    ]
                )

                system_msg = SystemMessage(system_prompt)

                # LLM inference
                try:
                    response: Step2Output = await structured_model.ainvoke(
                        [system_msg, human_msg]
                    )
                except Exception as llm_error:
                    dataset.at[idx, "error"] = f"LLM_ERROR: {llm_error}"
                    return
                
                response_json = response.model_dump()

                logger.info(
                            "LLM_RESPONSE | row=%s | response=%s",
                            idx,
                            response_json,
                            )

                expr = response.expression
                explanation = response.explanation


                if response.status != "ok" or not response.expression:
                    dataset.at[idx, "error"] = "LLM_ERROR: no experssion generated"
                    dataset.at[idx, "explanation"] = explanation
                    
                    return

                # Execute Math Expr.
                try:
                    result = execute_llm_expression(
                        raw_expression=expr,
                        variables={"price": extracted_price},
                    )

                    
                except Exception as exec_error:
                    dataset.at[idx, "error"] = f"EXECUTION_ERROR: {exec_error}"
                    dataset.at[idx, "final_expression"] = expr
                    dataset.at[idx, "explanation"] = explanation
                    return

               
                dataset.at[idx, "final_unit_price"] = result
                dataset.at[idx, "final_expression"] = expr
                dataset.at[idx, "explanation"] = explanation

            except Exception as fatal_error:
                dataset.at[idx, "error"] = f"FATAL_ERROR: {fatal_error}"

    #run tasks concurrently
    tasks = [
            process_row(idx, row)
            for idx, row in dataset.iterrows()
        ]

    await asyncio.gather(*tasks)

    return dataset


In [ ]:
# load dataset for testing
import pandas as pd
dataset = pd.read_excel('./dataset/dataset.xlsx')
dataset = dataset.rename(columns={'Unit\n':'TargetUnit','Rate £':'ExtractedPrice'})


In [ ]:
# Version 1 : 
output = await run_step2_and_3_on_dataset(
    dataset=dataset[-5:],
    structured_model=structured_model,
    system_prompt=SYSTEM_PROMPT,
    semaphore_limit=5,
)


In [ ]:
# Version 2 : 
output = await run_step2_and_3_on_dataset_v2(
    dataset=dataset[:5],
    structured_model=structured_model,
    semaphore_limit=5,
)

In [ ]:
output

In [ ]:
# save results
output.to_excel("./result/step2+3_output.xlsx")